# Set up Library

In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import MinMaxScaler ,StandardScaler ,OneHotEncoder
from sklearn.feature_selection import SelectKBest,f_classif, chi2,SelectFromModel,RFE
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_val_score,train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Preparation

In [3]:
data=pd.read_csv("file/Telco-Customer-Churn.csv")
df=pd.DataFrame(data)

In [4]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [5]:
df['Churn'].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [6]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

## Encode Data

In [7]:
#แทน Nan ด้วย Mean ของ Total Charges ทั้งหมด
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].mean())

#แทน Nan ด้วย 0
#df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)

df['Churn']=df['Churn'].replace({'No': 0, 'Yes': 1})
df['gender']=df['gender'].replace({'Male': 0, 'Female': 1})
df['Partner']=df['Partner'].replace({'No': 0, 'Yes': 1})
df['Dependents']=df['Dependents'].replace({'No': 0, 'Yes': 1})
df['PhoneService']=df['PhoneService'].replace({'No': 0, 'Yes': 1})
df['PaperlessBilling']=df['PaperlessBilling'].replace({'No': 0, 'Yes': 1})
df_select=df.drop(columns=['customerID'])
df_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int64  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int64  
 3   Dependents        7043 non-null   int64  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int64  
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   int64  
 16  PaymentMethod     7043 non-null   object 


C:\Users\Admin\AppData\Local\Temp\ipykernel_15320\131150282.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn']=df['Churn'].replace({'No': 0, 'Yes': 1})
C:\Users\Admin\AppData\Local\Temp\ipykernel_15320\131150282.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender']=df['gender'].replace({'Male': 0, 'Female': 1})
C:\Users\Admin\AppData\Local\Temp\ipykernel_15320\131150282.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, 

## df_select (Data Encoded Yes No)

In [8]:
df_select

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,0,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0
2,0,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,0,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,1,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,24,1,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,1,Mailed check,84.80,1990.50,0
7039,1,0,1,1,72,1,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,1,Credit card (automatic),103.20,7362.90,0
7040,1,0,1,1,11,0,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,1,Electronic check,29.60,346.45,0
7041,0,1,1,0,4,1,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Mailed check,74.40,306.60,1


## One Hot Encoding 

In [9]:
# เลือกคอลัมน์ที่เป็น Categorical (object) และ Numerical (ตัวเลข)
categorical_cols = df_select.select_dtypes(include=['object']).columns
numerical_cols = df_select.select_dtypes(include=['int64', 'float64']).columns

# เก็บ DataFrame ส่วนที่เป็น Numerical ไว้
df_select_numerical = df_select[numerical_cols]

# สร้าง OneHotEncoder instance
#เพิ่ม sparse_output=False เข้าไปใน OneHotEncoder เพื่อให้ผลลัพธ์ที่ได้เป็น NumPy array ทันที ซึ่งจะทำให้เราสามารถแปลงเป็น DataFrame ได้ง่ายขึ้น
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# ใช้ .fit_transform() เพื่อแปลงข้อมูลในคอลัมน์ที่เลือก
# เราต้องใส่ df_select[categorical_cols] เพราะต้องการให้ encoder เรียนรู้จากข้อมูลจริงใน DataFrame
transformed_data = encoder.fit_transform(df_select[categorical_cols])

# ดึงชื่อคอลัมน์ใหม่ที่ถูกสร้างขึ้นมา
new_column_names = encoder.get_feature_names_out(categorical_cols)

# สร้าง DataFrame ใหม่จากข้อมูลที่แปลงแล้วและชื่อคอลัมน์ใหม่
df_encoded = pd.DataFrame(data=transformed_data, columns=new_column_names)

# รวม DataFrame ที่แปลงแล้วเข้ากับ DataFrame เดิมที่เป็นตัวเลข
new_df = pd.concat([df_select_numerical, df_encoded], axis=1)
new_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 41 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   gender                                   7043 non-null   int64  
 1   SeniorCitizen                            7043 non-null   int64  
 2   Partner                                  7043 non-null   int64  
 3   Dependents                               7043 non-null   int64  
 4   tenure                                   7043 non-null   int64  
 5   PhoneService                             7043 non-null   int64  
 6   PaperlessBilling                         7043 non-null   int64  
 7   MonthlyCharges                           7043 non-null   float64
 8   TotalCharges                             7043 non-null   float64
 9   Churn                                    7043 non-null   int64  
 10  MultipleLines_No                         7043 no

# new_df (Complete Encoded Data)

In [10]:
new_df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,1,29.85,29.85,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0,0,0,34,1,0,56.95,1889.50,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0,0,0,0,2,1,1,53.85,108.15,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,0,0,0,45,0,0,42.30,1840.75,0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1,0,0,0,2,1,1,70.70,151.65,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,24,1,1,84.80,1990.50,0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7039,1,0,1,1,72,1,1,103.20,7362.90,0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7040,1,0,1,1,11,0,1,29.60,346.45,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7041,0,1,1,0,4,1,1,74.40,306.60,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


# export data to csv 

In [ ]:
new_df.to_csv('prepare_data.csv', index=False)

## Train Validation Test split data

In [11]:
#ทำdata split เพื่อแบ่งไว้เรียนรู้และทดสอบ จะได้ไม่โกง
from sklearn.model_selection import train_test_split

# สมมติว่า X เป็นข้อมูลคุณลักษณะ (features) และ y เป็นข้อมูลเป้าหมาย (target)
X = new_df.drop(columns=['Churn'])
y = new_df['Churn']

# --- ขั้นตอนที่ 1: แบ่งข้อมูลทั้งหมด (100%) ออกเป็น Train (60%) และ Temp (40%) ---
# แบ่งข้อมูลเป็นชุดเทรนและชุดทดสอบ
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42,stratify=y)

# --- ขั้นตอนที่ 2: แบ่ง Temp (40%) ออกเป็น Validation (20%) และ Test (20%) ---
# ตอนนี้เรามี X_temp 40% เราต้องแบ่งครึ่ง (0.5) เพื่อให้ได้ 20% กับ 20%
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 50% ของ 40% = 20% ของข้อมูลทั้งหมด
    random_state=42,
    stratify=y_temp # รักษาสัดส่วนของ y_temp ด้วย
)
print(f"ข้อมูลทั้งหมด: {len(X)}")
print(f"Train set:    {len(X_train)} ({(len(X_train)/len(X))*100:.0f}%)")
print(f"Validation set: {len(X_val)} ({(len(X_val)/len(X))*100:.0f}%)")
print(f"Test set:     {len(X_test)} ({(len(X_test)/len(X))*100:.0f}%)")

ข้อมูลทั้งหมด: 7043
Train set:    4225 (60%)
Validation set: 1409 (20%)
Test set:     1409 (20%)


# Export Train, Validation, Test to csv

In [20]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df.to_csv('train_df.csv', index=False)

In [21]:
val_df = pd.concat([X_val, y_val], axis=1)
val_df.to_csv('val_df.csv', index=False)

In [22]:
test_df = pd.concat([X_test, y_test], axis=1)
test_df.to_csv('test_df.csv', index=False)